# Compile the results on the MOF case study 

In [31]:
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
plt.style.use(['science', 'nature'])
import pandas as pd 
from fastcore.helpers import load_pickle
from gpt3forchem.helpers import get_else_nan

## QMOF 

### Classification 

In [32]:
all_qmof_classification_res = glob('/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220913_mof_classification/*.pkl')

In [33]:
compiled_qmof_classification_data = []
compiled_qmof_baseline_data = []

for filename in all_qmof_classification_res:
    loaded_res = load_pickle(filename)

    res = {
        "accuracy": get_else_nan(loaded_res["cm"], "ACC_Macro"),
        "f1_macro": get_else_nan(loaded_res["cm"], "F1_Macro"),
        "f1_micro": get_else_nan(loaded_res["cm"], "F1_Micro"),
        "train_size": loaded_res["train_size"],
        "mcc": get_else_nan(loaded_res["cm"], "Overall_MCC"),
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    baseline_res = {
        "baseline_accuracy": get_else_nan(loaded_res["baseline_cm"], "ACC_Macro"),
        "baseline_f1_macro": get_else_nan(loaded_res["baseline_cm"], "F1_Macro"),
        "baseline_f1_micro": get_else_nan(loaded_res["baseline_cm"], "F1_Micro"),
        "baseline_mcc": get_else_nan(loaded_res["baseline_cm"], "Overall_MCC"),
        "train_size": loaded_res["train_size"],
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    compiled_qmof_classification_data.append(res)
    compiled_qmof_baseline_data.append(baseline_res)


In [34]:
compiled_qmof_classification_data = pd.DataFrame(compiled_qmof_classification_data)
compiled_qmof_baseline_data = pd.DataFrame(compiled_qmof_baseline_data)

In [35]:
compiled_qmof_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_50323/4159097433.py:1: FutureWarning: ['mcc'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_qmof_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])


accuracy            \
                                                               mean       std   
target                  representation         train_size                       
outputs.pbe.bandgap_cat chemical_name          10          0.735467  0.021914   
                                               50          0.768800  0.041899   
                                               100         0.818133  0.014048   
                                               200         0.836400  0.003960   
                                               500         0.855200       NaN   
                                               1000        0.879533  0.018573   
                                               2000        0.889200  0.003960   
                                               3000        0.922400       NaN   
                        info.mofid.mofid_clean 10          0.717433  0.037845   
                                               50          0.763985  0.024521   
                                               100         0.845259  0.058360   
                                               200         0.830480  0.046759   
                                               500         0.870667  0.007668   
                                               1000        0.883387  0.007723   
                                               2000        0.900533  0.009272   
                                               3000        0.908000  0.015839   

                                                           f1_macro            \
                                                               mean       std   
target                  representation         train_size                       
outputs.pbe.bandgap_cat chemical_name          10          0.194741  0.038253   
                                               50          0.182454  0.098772   
                                               100         0.292088  0.052740   
                                               200         0.360514  0.014498   
                                               500         0.462298       NaN   
                                               1000        0.469777  0.040520   
                                               2000        0.608783  0.064483   
                                               3000        0.621715       NaN   
                        info.mofid.mofid_clean 10          0.141765  0.090571   
                                               50          0.174914  0.070117   
                                               100         0.293975  0.158088   
                                               200         0.368876  0.096230   
                                               500         0.501428  0.014631   
                                               1000        0.512295  0.044672   
                                               2000        0.577524  0.123864   
                                               3000        0.605338  0.035133   

                                                           f1_micro            
                                                               mean       std  
target                  representation         train_size                      
outputs.pbe.bandgap_cat chemical_name          10          0.338667  0.054784  
                                               50          0.422000  0.104747  
                                               100         0.545333  0.035119  
                                               200         0.591000  0.009899  
                                               500         0.638000       NaN  
                                               1000        0.672000  0.008485  
                                               2000        0.723000  0.009899  
                                               3000        0.806000       NaN  
                        info.mofid.mofid_clean 10          0.252500  0.168312  
                   

In [36]:
compiled_qmof_baseline_data.groupby(['target', 'representation', 'train_size']).agg(['std', 'mean'])

baseline_accuracy  \
                                                                        std   
target                  representation         train_size                     
outputs.pbe.bandgap_cat chemical_name          10                       NaN   
                                               50                       NaN   
                                               100                      NaN   
                                               200                      NaN   
                                               500                      NaN   
                                               1000                     NaN   
                                               2000                     NaN   
                                               3000                     NaN   
                        info.mofid.mofid_clean 10                       NaN   
                                               50                       NaN   
                                               100                      NaN   
                                               200                      NaN   
                                               500                      NaN   
                                               1000                     NaN   
                                               2000                     NaN   
                                               3000                     NaN   

                                                                     \
                                                               mean   
target                  representation         train_size             
outputs.pbe.bandgap_cat chemical_name          10               NaN   
                                               50          0.777778   
                                               100              NaN   
                                               200              NaN   
                                               500              NaN   
                                               1000             NaN   
                                               2000             NaN   
                                               3000             NaN   
                        info.mofid.mofid_clean 10               NaN   
                                               50               NaN   
                                               100              NaN   
                                               200              NaN   
                                               500              NaN   
                                               1000             NaN   
                                               2000             NaN   
                                               3000             NaN   

                                                          baseline_f1_macro  \
                                                                        std   
target                  representation         train_size                     
outputs.pbe.bandgap_cat chemical_name          10                       NaN   
                                               50                       NaN   
                                               100                      NaN   
                                               200                      NaN   
                                               500                      NaN   
                                               1000                     NaN   
                                               2000                     NaN   
                                               3000                     NaN   
                        info.mofid.mofid_clean 10                       NaN   
                                               50                       NaN   
                                               100                      NaN   
                                               200                      NaN   
                     

### Regression 

In [37]:
qmof_regression_results = glob('/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/*.pkl')

In [38]:
load_pickle(qmof_regression_results[0])

{'model_type': 'ada',
 'train_set_size': 2000,
 'prefix': '',
 'train_size': 2000,
 'test_size': 1627,
 'metrics': {'r2': 0.7192669331671564,
  'max_error': 2.5827999999999998,
  'mean_absolute_error': 0.435709634,
  'mean_squared_error': 0.374205485176178},
 'completions': {'choices': [<OpenAIObject at 0x16cc5bcc0> JSON: {
     "finish_reason": "length",
     "index": 0,
     "logprobs": null,
     "text": " 0.9595@@@4.936"
   },
   <OpenAIObject at 0x16cc8a4a0> JSON: {
     "finish_reason": "length",
     "index": 1,
     "logprobs": null,
     "text": " 1.934@@@+1.9"
   },
   <OpenAIObject at 0x295acb680> JSON: {
     "finish_reason": "length",
     "index": 2,
     "logprobs": null,
     "text": " 0.49@@@@@@ 0.49"
   },
   <OpenAIObject at 0x295acbd10> JSON: {
     "finish_reason": "length",
     "index": 3,
     "logprobs": null,
     "text": " 0.03934@@@0.49"
   },
   <OpenAIObject at 0x295acb450> JSON: {
     "finish_reason": "length",
     "index": 4,
     "logprobs": null,
   

In [39]:
compiled_regression_results = []
compiled_regression_baseline_results = []

for res_file in qmof_regression_results:
    try:
        res = load_pickle(res_file)
        res_dict = {
            'r2': res['metrics']['r2'],
            'max_error': res['metrics']['max_error'],
            'mean_absolute_error': res['metrics']['mean_absolute_error'],
            'mean_squared_error': res['metrics']['mean_squared_error'],
        }
        res_dict['train_size'] = res['train_size']
        res_dict['representation'] = res['representation']
        res_dict['target'] = res['target']
    except Exception: 
        pass
    baseline_res_dict = res['baseline_metrics']
    baseline_res_dict['train_size'] = res['train_size']
    baseline_res_dict['target'] = res['target']

    compiled_regression_results.append(res_dict)
    compiled_regression_baseline_results.append(baseline_res_dict)
    

In [40]:
compiled_regression_results = pd.DataFrame(compiled_regression_results)
compiled_regression_baseline_results = pd.DataFrame(compiled_regression_baseline_results)

In [41]:
compiled_regression_results.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

r2            \
                                                           mean       std   
target              representation         train_size                       
outputs.pbe.bandgap chemical_name          10         -0.443025       NaN   
                                           50         -1.133713       NaN   
                                           100         0.157699       NaN   
                                           200         0.434276       NaN   
                                           1000        0.613104       NaN   
                    info.mofid.mofid_clean 10         -0.386953  0.017388   
                                           50         -0.193842  0.276091   
                                           100        -0.391992  0.491365   
                                           200         0.385412  0.091779   
                                           500         0.587263  0.039966   
                                           1000        0.543437  0.096453   
                                           2000        0.652935  0.085578   
                                           3000        0.699638  0.020831   

                                                      max_error            \
                                                           mean       std   
target              representation         train_size                       
outputs.pbe.bandgap chemical_name          10          2.918800       NaN   
                                           50          4.822000       NaN   
                                           100         2.961480       NaN   
                                           200         2.754800       NaN   
                                           1000        2.851600       NaN   
                    info.mofid.mofid_clean 10          3.491233  0.229320   
                                           50          3.409937  0.633929   
                                           100         3.858023  0.534190   
                                           200         3.638514  0.534815   
                                           500         3.075750  0.722592   
                                           1000        3.088475  0.020962   
                                           2000        2.741890  0.186622   
                                           3000        2.943025  0.208280   

                                                      mean_absolute_error  \
                                                                     mean   
target              representation         train_size                       
outputs.pbe.bandgap chemical_name          10                    1.107028   
                                           50                    1.416127   
                                           100                   0.845537   
                                           200                   0.665363   
                                           1000                  0.564574   
                    info.mofid.mofid_clean 10                    1.110821   
                                           50                    0.997523   
                                           100                   1.066413   
                                           200                   0.699816   
                                           500                   0.550163   
                                           1000                  0.584356   
                                           2000                  0.492304   
                                           3000                  0.467493   

                                                                 \
                                                            std   
target              representation         train_size             
outputs.pbe.bandgap chemical_name          10               NaN   
                                           50               NaN   
                           

In [42]:
compiled_regression_baseline_results.groupby(['target', 'train_size']).agg(['mean', 'std'])

r2     max_error      \
                                    mean std      mean std   
target              train_size                               
outputs.pbe.bandgap 10         -1.858278 NaN  5.688000 NaN   
                    50         -0.136081 NaN  4.494358 NaN   
                    100        -0.175163 NaN  4.317953 NaN   
                    200        -0.165965 NaN  4.177559 NaN   
                    500        -0.197219 NaN  4.479406 NaN   
                    1000       -0.140141 NaN  4.219677 NaN   
                    2000       -0.142327 NaN  3.987293 NaN   
                    3000             NaN NaN       NaN NaN   

                               mean_absolute_error     mean_squared_error      
                                              mean std               mean std  
target              train_size                                                 
outputs.pbe.bandgap 10                    1.644007 NaN           3.945707 NaN  
                    50                    1.047463 NaN           1.563870 NaN  
                    100                   1.063629 NaN           1.617324 NaN  
                    200                   1.043849 NaN           1.589083 NaN  
                    500                   1.067150 NaN           1.657924 NaN  
                    1000                  1.050336 NaN           1.575476 NaN  
                    2000                  1.051185 NaN           1.569617 NaN  
                    3000                       NaN NaN                NaN NaN

## CoRE MOF

In [43]:
all_core_results = glob('results/20220914_mof_classification/*pkl')

In [44]:
compiled_core_classification_data = []
compiled_core_baseline_data = []

for filename in all_core_results:
    loaded_res = load_pickle(filename)

    res = {
        "accuracy": get_else_nan(loaded_res["cm"], "ACC_Macro"),
        "f1_macro": get_else_nan(loaded_res["cm"], "F1_Macro"),
        "f1_micro": get_else_nan(loaded_res["cm"], "F1_Micro"),
        "train_size": loaded_res["train_size"],
        "mcc": get_else_nan(loaded_res["cm"], "Overall_MCC"),
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    baseline_res = {
        "baseline_accuracy": get_else_nan(loaded_res["baseline_cm"], "ACC_Macro"),
        "baseline_f1_macro": get_else_nan(loaded_res["baseline_cm"], "F1_Macro"),
        "baseline_f1_micro": get_else_nan(loaded_res["baseline_cm"], "F1_Micro"),
        "baseline_mcc": get_else_nan(loaded_res["baseline_cm"], "Overall_MCC"),
        "train_size": loaded_res["train_size"],
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    compiled_core_classification_data.append(res)
    compiled_core_baseline_data.append(baseline_res)


In [45]:
compiled_core_classification_data = pd.DataFrame(compiled_core_classification_data)

In [46]:
compiled_core_classification_data_agg = compiled_core_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_50323/2870106362.py:1: FutureWarning: ['mcc'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_core_classification_data_agg = compiled_core_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])


In [47]:
compiled_core_classification_data_agg

accuracy            \
                                                      mean       std   
target                representation  train_size                       
outputs.CH4DC_cat     chemical_name_y 10          0.717800  0.010701   
                                      50          0.727200  0.005581   
                                      100         0.721400  0.006000   
                                      200         0.806800  0.124025   
                                      500         0.736533  0.005478   
                                      1000        0.773067  0.008992   
                      clean_mofid     10          0.745229  0.043210   
                                      50          0.720600  0.008941   
                                      100         0.719733  0.002013   
                                      200         0.736400  0.014816   
                                      500         0.773462  0.059603   
                                      1000        0.756800  0.006882   
outputs.logKH_CO2_cat chemical_name_y 10          0.778433  0.006467   
                                      50          0.771600  0.006450   
                                      100         0.768800  0.012445   
                                      200         0.777000  0.007016   
                                      500         0.790175  0.020426   
                                      1000        0.837956  0.016460   
                      clean_mofid     10          0.764833  0.036884   
                                      50          0.772000  0.007811   
                                      100         0.768000  0.008641   
                                      200         0.782000  0.011323   
                                      500         0.790200  0.005362   
                                      1000        0.803400  0.010681   

                                                  f1_macro            \
                                                      mean       std   
target                representation  train_size                       
outputs.CH4DC_cat     chemical_name_y 10          0.149823  0.027937   
                                      50          0.130674  0.023026   
                                      100         0.153043  0.016027   
                                      200         0.132933  0.123551   
                                      500         0.249662  0.076365   
                                      1000        0.357930  0.034869   
                      clean_mofid     10          0.124279  0.026044   
                                      50          0.159902  0.047351   
                                      100         0.135585  0.020836   
                                      200         0.198593  0.056259   
                                      500         0.210406  0.108564   
                                      1000        0.310452  0.026149   
outputs.logKH_CO2_cat chemical_name_y 10          0.131826  0.009319   
                                      50          0.125988  0.010818   
                                      100         0.156510  0.024739   
                                      200         0.199429  0.050453   
                                      500         0.259241  0.153433   
                                      1000        0.327946  0.103090   
                      clean_mofid     10          0.163316  0.027357   
                                      50          0.168882  0.034863   
                                      100         0.162777  0.025103   
                                      200         0.208806  0.030507   
                                      500         0.224695  0.033614   
                                      1000        0.303062  0.038744   

                                                  f1_micro            
                                                      mean       std  
target            

In [48]:
compiled_core_baseline_data = pd.DataFrame(compiled_core_baseline_data)

In [50]:
compiled_core_baseline_data.groupby(['target', 'representation', "train_size"]).agg(['mean', 'std'])

baseline_accuracy  \
                                                              mean   
target                representation  train_size                     
outputs.CH4DC_cat     chemical_name_y 10                       NaN   
                                      50                       NaN   
                                      100                      NaN   
                                      200                      NaN   
                                      500                      NaN   
                                      1000                     NaN   
                      clean_mofid     10                       NaN   
                                      50                  0.800000   
                                      100                 0.800000   
                                      200                 0.800000   
                                      500                      NaN   
                                      1000                     NaN   
outputs.logKH_CO2_cat chemical_name_y 10                       NaN   
                                      50                       NaN   
                                      100                      NaN   
                                      200                      NaN   
                                      500                      NaN   
                                      1000                     NaN   
                      clean_mofid     10                       NaN   
                                      50                       NaN   
                                      100                      NaN   
                                      200                 0.788889   
                                      500                      NaN   
                                      1000                     NaN   

                                                                \
                                                           std   
target                representation  train_size                 
outputs.CH4DC_cat     chemical_name_y 10                   NaN   
                                      50                   NaN   
                                      100                  NaN   
                                      200                  NaN   
                                      500                  NaN   
                                      1000                 NaN   
                      clean_mofid     10                   NaN   
                                      50          0.000000e+00   
                                      100         1.110223e-16   
                                      200                  NaN   
                                      500                  NaN   
                                      1000                 NaN   
outputs.logKH_CO2_cat chemical_name_y 10                   NaN   
                                      50                   NaN   
                                      100                  NaN   
                                      200                  NaN   
                                      500                  NaN   
                                      1000                 NaN   
                      clean_mofid     10                   NaN   
                                      50                   NaN   
                                      100                  NaN   
                                      200         1.571348e-02   
                                      500                  NaN   
                                      1000                 NaN   

                                                 baseline_f1_macro       \
                                                              mean  std   
target                representation  train_size                          
outputs.CH4DC_cat     chemical_name_y 10                       NaN  NaN   
                                      50                       NaN  NaN   
